<a href="https://colab.research.google.com/github/alekzanderx1/NYC-Citywide-Payroll-Data-Profiling-and-Cleaning/blob/main/DataCleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Load Data

Here we will clean the data based on findings from the previous notebook where we did data profiling. 

We will start with loading the data through Socrata and converting it to a dataframe to make necessary modifications.

In [1]:
pip install openclean

     |████████████████████████████████| 267 kB 7.0 MB/s 
     |████████████████████████████████| 69 kB 6.4 MB/s 
     |████████████████████████████████| 109 kB 50.5 MB/s 
     |████████████████████████████████| 260 kB 53.2 MB/s 
     |████████████████████████████████| 137 kB 45.2 MB/s 
     |████████████████████████████████| 596 kB 37.0 MB/s 
     |████████████████████████████████| 525 kB 44.9 MB/s 
     |████████████████████████████████| 180 kB 41.9 MB/s 
     |████████████████████████████████| 206 kB 23.0 MB/s 
     |████████████████████████████████| 149 kB 70.7 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 961 kB 59.9 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
     |████████████████████████████████| 3.5 MB 58.5 MB/s 
     |████████████████████████████████| 636 kB 31.6 MB/s 
  Created wheel for datasize: filename=datasize-1.0.0-py2.py3-none-any.whl size=155047 sha256=eee93afca09f1900d01171900bda531222b7453b2cb

In [2]:
from datetime import datetime
from openclean.function.eval.base import Col
from openclean.function.eval.datatype import Datetime
from openclean.function.eval.logic import And
from openclean.function.eval.null import IsNotEmpty
from openclean.function.eval.domain import Lookup
from openclean.data.source.socrata import Socrata
import gzip
import os

dataset = Socrata().dataset('k397-673e')

In [3]:
datafile = './k397-673e.tsv.gz'

In [4]:
if not os.path.isfile(datafile):
    with gzip.open(datafile, 'wb') as f:
        print('Downloading ...\n')
        dataset.write(f)

In [5]:
from openclean.pipeline import stream

ds_full = stream(datafile)

In [6]:
ds_full.count()

4496767

## Removing unwanted columns

We are removing 'Payroll Number', 'First Name', 'Last Name', and 'Mid Init' because we will not be using them in our analysis going forwards.

In [7]:
columns=[
    'Fiscal Year',
    'Agency Name',
    'Agency Start Date',
    'Work Location Borough',
    'Title Description',
    'Leave Status as of June 30',
    'Base Salary',
    'Pay Basis',
    'Regular Hours',
    'Regular Gross Paid',
    'OT Hours',
    'Total OT Paid',
    'Total Other Pay'
]
ds_full = ds_full.select(columns)

In [8]:
# convert to datafrane

df = ds_full.to_df()

In [9]:
df.shape

(4496767, 13)

## Clean Borough

We will be -
1. Updating to uppercase
2. Replacing empty string '' to UNSPECIFIED
3. Filter on all borough names
4. Rename 'RICHMOND' to 'STATEN ISLAND'

In [10]:
from openclean.operator.transform.filter import delete


In [11]:
df.shape

(4496767, 13)

In [12]:
# Update to uppercase

from openclean.operator.transform.update import update

df = update(df, columns='Work Location Borough', func=str.upper)

In [13]:
df['Work Location Borough'].unique()

array(['BROOKLYN', 'MANHATTAN', 'BRONX', 'RICHMOND', 'QUEENS',
       'WASHINGTON DC', 'ULSTER', 'WESTCHESTER', 'OTHER', 'ALBANY',
       'NASSAU', 'DELAWARE', 'SULLIVAN', '', 'ORANGE', 'SCHOHARIE',
       'DUTCHESS', 'GREENE', 'PUTNAM'], dtype=object)

In [14]:
# Replace empty string '' with 'UNSPECIFIED'

df=update(df, columns='Work Location Borough',func=lambda x: x if x != '' else 'UNSPECIFIED')

In [15]:
df['Work Location Borough'].unique()

array(['BROOKLYN', 'MANHATTAN', 'BRONX', 'RICHMOND', 'QUEENS',
       'WASHINGTON DC', 'ULSTER', 'WESTCHESTER', 'OTHER', 'ALBANY',
       'NASSAU', 'DELAWARE', 'SULLIVAN', 'UNSPECIFIED', 'ORANGE',
       'SCHOHARIE', 'DUTCHESS', 'GREENE', 'PUTNAM'], dtype=object)

In [16]:
# Keep just the 5 boroughs and 'UNSPECIFIED'

df = df[(df['Work Location Borough'] == 'BROOKLYN') | \
        (df['Work Location Borough'] == 'MANHATTAN') | \
        (df['Work Location Borough'] == 'BRONX') | \
        (df['Work Location Borough'] == 'RICHMOND') | \
        (df['Work Location Borough'] == 'QUEENS') | \
        (df['Work Location Borough'] == 'UNSPECIFIED')]

In [17]:
df['Work Location Borough'].unique()

array(['BROOKLYN', 'MANHATTAN', 'BRONX', 'RICHMOND', 'QUEENS',
       'UNSPECIFIED'], dtype=object)

In [18]:
df.shape

(4392154, 13)

In this column, there are 52,803 instances of 'RICHMOND'. For the purposes of clarity, we decided to manually rename these to 'STATEN ISLAND'

In [19]:
df[df['Work Location Borough'] == 'RICHMOND'].count

<bound method DataFrame.count of         Fiscal Year  ... Total Other Pay
741            2020  ...         1000.00
759            2020  ...          375.00
829            2020  ...            0.00
966            2020  ...          375.00
1038           2020  ...            0.00
...             ...  ...             ...
4495992        2021  ...           -4.59
4496206        2021  ...         -377.24
4496437        2021  ...        -4369.96
4496607        2021  ...         4191.08
4496616        2021  ...        -9100.90

[53766 rows x 13 columns]>

In [20]:
df['Work Location Borough'].unique()

array(['BROOKLYN', 'MANHATTAN', 'BRONX', 'RICHMOND', 'QUEENS',
       'UNSPECIFIED'], dtype=object)

In [21]:
# Rename 'RICHMOND' to 'STATEN ISLAND'

df = update(df, columns='Work Location Borough', func= lambda x : 'STATEN ISLAND' if x == 'RICHMOND' else x)

In [22]:
df['Work Location Borough'].unique()

array(['BROOKLYN', 'MANHATTAN', 'BRONX', 'STATEN ISLAND', 'QUEENS',
       'UNSPECIFIED'], dtype=object)

## Converting Columns to Appropriate Datatypes

In [23]:
df = df.astype({"Fiscal Year" : int, "Base Salary": float, "Regular Hours": float, "Regular Gross Paid": float, "OT Hours": float, "Total OT Paid": float, "Total Other Pay": float})


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4392154 entries, 0 to 4496766
Data columns (total 13 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   Fiscal Year                 int64  
 1   Agency Name                 object 
 2   Agency Start Date           object 
 3   Work Location Borough       object 
 4   Title Description           object 
 5   Leave Status as of June 30  object 
 6   Base Salary                 float64
 7   Pay Basis                   object 
 8   Regular Hours               float64
 9   Regular Gross Paid          float64
 10  OT Hours                    float64
 11  Total OT Paid               float64
 12  Total Other Pay             float64
dtypes: float64(6), int64(1), object(6)
memory usage: 469.1+ MB


## Cleanup Negative Values from Numerical Columns

First we will check the count of rows which have negative values and remove them at the end.

In [25]:
df[df['Regular Gross Paid'] < 0]

,Fiscal Year,Agency Name,Agency Start Date,Work Location Borough,Title Description,Leave Status as of June 30,Base Salary,Pay Basis,Regular Hours,Regular Gross Paid,OT Hours,Total OT Paid,Total Other Pay
2902,2020,LAW DEPARTMENT,12/08/2008,MANHATTAN,PARALEGAL AIDE,CEASED,49677.00,per Annum,0.0,-52.86,7.25,209.38,8.10
2903,2020,LAW DEPARTMENT,10/15/2017,MANHATTAN,CLAIM SPECIALIST,CEASED,40275.00,per Annum,0.0,-5.52,7.00,162.58,0.00
2907,2020,LAW DEPARTMENT,12/16/2013,MANHATTAN,CLAIM SPECIALIST,CEASED,44409.00,per Annum,0.0,-70.15,9.75,195.20,9.51
2916,2020,LAW DEPARTMENT,06/06/2016,BROOKLYN,CLERICAL ASSOCIATE,ON LEAVE,41848.00,per Annum,0.0,-0.01,3.00,56.18,0.00
2921,2020,LAW DEPARTMENT,10/22/2018,MANHATTAN,SENIOR STUDENT LEGAL SPECIALIST,CEASED,49157.00,per Annum,-70.0,-711.63,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4496758,2021,DEPT OF ED HRLY SUPPORT STAFF,04/27/1999,MANHATTAN,F/T SCHOOL AIDE,CEASED,17.00,per Hour,0.0,-57256.00,0.00,0.00,5814.70
4496759,2021,DEPT OF ED PEDAGOGICAL,09/06/2005,MANHATTAN,ASSISTANT PRINCIPAL,ON LEAVE,130351.00,per Annum,0.0,-36364.44,0.00,0.00,-15369.52
4496760,2021,DEPT OF ED HRLY SUPPORT STAFF,09/02/1997,MANHATTAN,F/T SCHOOL AIDE,CEASED,17.04,per Hour,0.0,-58284.17,0.00,0.00,4347.24
4496763,2021,BRONX DISTRICT ATTORNEY,07/02/1990,BRONX,SPECIAL ASSISTANT TO THE DISTRICT ATTORNEY,CEASED,110000.00,per Annum,-70.0,-4207.65,0.00,0.00,-75440.00


In [26]:
df[df['Total OT Paid'] < 0]

,Fiscal Year,Agency Name,Agency Start Date,Work Location Borough,Title Description,Leave Status as of June 30,Base Salary,Pay Basis,Regular Hours,Regular Gross Paid,OT Hours,Total OT Paid,Total Other Pay
39015,2020,POLICE DEPARTMENT,02/05/1981,BRONX,PRINCIPAL ADMINISTRATIVE ASSOCIATE - NON SUPVR,ACTIVE,64124.0,per Annum,1820.00,65275.97,-19.57,-1001.20,5882.60
49840,2020,POLICE DEPARTMENT,04/23/2007,MANHATTAN,POLICE ADMINISTRATIVE AIDE,ACTIVE,47385.0,per Annum,1820.00,45243.72,-1.00,-24.93,6650.42
53568,2020,POLICE DEPARTMENT,09/27/2012,MANHATTAN,TRAFFIC ENFORCEMENT AGENT,ACTIVE,42377.0,per Annum,1929.50,41369.48,-2.00,-57.79,2238.91
55147,2020,POLICE DEPARTMENT,07/10/2002,MANHATTAN,POLICE ADMINISTRATIVE AIDE,ACTIVE,41988.0,per Annum,1638.00,33179.40,-1.00,-25.01,4695.83
55990,2020,POLICE DEPARTMENT,11/14/2005,BROOKLYN,SCHOOL CROSSING GUARD,ACTIVE,34626.0,per Annum,2080.00,34094.19,-5.00,-42.67,758.92
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4496544,2021,ADMIN FOR CHILDREN'S SVCS,09/23/2019,BROOKLYN,YOUTH DEVELOPMENT SPECIALIST,CEASED,45759.0,per Annum,-280.00,-6163.62,-8.00,-200.37,-52.60
4496573,2021,DEPARTMENT OF SANITATION,04/03/1989,BROOKLYN,SANITATION WORKER,CEASED,81034.0,per Annum,0.00,0.00,-2.00,-39.09,-7504.64
4496634,2021,ADMIN FOR CHILDREN'S SVCS,02/25/2019,BROOKLYN,YOUTH DEVELOPMENT SPECIALIST,CEASED,49318.0,per Annum,-420.00,-10287.97,-24.50,-755.83,-21.87
4496732,2021,DEPARTMENT OF SANITATION,05/03/1982,BROOKLYN,PRINCIPAL ADMINISTRATIVE ASSOCIATE - NON SUPVR,CEASED,53082.0,per Annum,0.00,-2264.19,0.00,-730.47,-24140.37


In [27]:
df[df['Total Other Pay'] < 0]

,Fiscal Year,Agency Name,Agency Start Date,Work Location Borough,Title Description,Leave Status as of June 30,Base Salary,Pay Basis,Regular Hours,Regular Gross Paid,OT Hours,Total OT Paid,Total Other Pay
5,2020,OFFICE OF EMERGENCY MANAGEMENT,03/18/2019,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.0,per Annum,1820.0,87900.95,0.00,0.00,-3202.74
646,2020,TAX COMMISSION,12/05/2010,MANHATTAN,SECRETARY OF THE TAX COMMISSION,ACTIVE,63709.0,per Annum,1820.0,62824.90,0.00,0.00,-61.62
1259,2020,LAW DEPARTMENT,09/12/2011,MANHATTAN,ASSISTANT CORPORATION COUNSEL,ACTIVE,101077.0,per Annum,1820.0,98390.54,0.00,0.00,-3919.32
1395,2020,LAW DEPARTMENT,09/11/2000,MANHATTAN,ASSISTANT CORPORATION COUNSEL,ACTIVE,91616.0,per Annum,1820.0,88485.80,0.00,0.00,-1805.36
1539,2020,LAW DEPARTMENT,06/18/2001,BRONX,ASSOCIATE QUALITY ASSURANCE SPECIALIST,ACTIVE,77169.0,per Annum,1820.0,78040.71,1.25,186.24,-72.39
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4496754,2021,DEPT OF ED PEDAGOGICAL,07/01/2005,MANHATTAN,PRINCIPAL,CEASED,148594.0,per Annum,0.0,0.00,0.00,0.00,-47536.59
4496755,2021,DEPT OF ED PEDAGOGICAL,07/01/2007,MANHATTAN,TEACHER SPECIAL EDUCATION,CEASED,98486.0,per Annum,0.0,-48341.88,0.00,0.00,-13.35
4496757,2021,DEPT OF ED PEDAGOGICAL,09/06/2016,MANHATTAN,TEACHER-REG SUB,CEASED,124909.0,per Annum,0.0,-51156.68,0.00,0.00,-16.93
4496759,2021,DEPT OF ED PEDAGOGICAL,09/06/2005,MANHATTAN,ASSISTANT PRINCIPAL,ON LEAVE,130351.0,per Annum,0.0,-36364.44,0.00,0.00,-15369.52


In [28]:
df[df['Regular Hours'] < 0]

,Fiscal Year,Agency Name,Agency Start Date,Work Location Borough,Title Description,Leave Status as of June 30,Base Salary,Pay Basis,Regular Hours,Regular Gross Paid,OT Hours,Total OT Paid,Total Other Pay
2921,2020,LAW DEPARTMENT,10/22/2018,MANHATTAN,SENIOR STUDENT LEGAL SPECIALIST,CEASED,49157.0,per Annum,-70.00,-711.63,0.0,0.00,0.00
57539,2020,POLICE DEPARTMENT,12/20/1998,MANHATTAN,SCHOOL SAFETY AGENT,ON LEAVE,50207.0,per Annum,-63.00,24806.12,0.0,0.00,252.79
61546,2020,POLICE DEPARTMENT,12/20/1998,MANHATTAN,SCHOOL SAFETY AGENT,CEASED,48745.0,per Annum,-40.00,12910.02,0.0,122.13,133.64
61719,2020,POLICE DEPARTMENT,09/25/2006,MANHATTAN,SCHOOL SAFETY AGENT,CEASED,48745.0,per Annum,-45.00,10633.94,0.0,0.00,1804.54
61733,2020,POLICE DEPARTMENT,07/01/2002,MANHATTAN,SCHOOL SAFETY AGENT,CEASED,48745.0,per Annum,-72.00,12435.60,0.0,0.00,-45.45
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4496703,2021,DEPARTMENT OF EDUCATION ADMIN,09/08/2015,BROOKLYN,PHYSICAL THERAPIST,ON LEAVE,78379.0,per Annum,-448.00,-20598.71,0.0,0.00,754.63
4496715,2021,DEPARTMENT OF CORRECTION,11/03/2005,QUEENS,CAPTAIN,CEASED,109360.0,per Annum,-56.00,-2927.19,0.0,0.00,-18831.37
4496729,2021,DEPARTMENT OF CORRECTION,05/16/2013,QUEENS,CORRECTION OFFICER,CEASED,89391.0,per Annum,-56.00,217.80,0.0,0.00,-26556.12
4496739,2021,FIRE DEPARTMENT,09/14/1987,BROOKLYN,FIREFIGHTER,CEASED,85292.0,per Annum,-5.72,-233.05,-132.0,-5022.29,-23835.73


In [29]:
df[df['OT Hours'] < 0]

,Fiscal Year,Agency Name,Agency Start Date,Work Location Borough,Title Description,Leave Status as of June 30,Base Salary,Pay Basis,Regular Hours,Regular Gross Paid,OT Hours,Total OT Paid,Total Other Pay
39015,2020,POLICE DEPARTMENT,02/05/1981,BRONX,PRINCIPAL ADMINISTRATIVE ASSOCIATE - NON SUPVR,ACTIVE,64124.0,per Annum,1820.00,65275.97,-19.57,-1001.20,5882.60
49840,2020,POLICE DEPARTMENT,04/23/2007,MANHATTAN,POLICE ADMINISTRATIVE AIDE,ACTIVE,47385.0,per Annum,1820.00,45243.72,-1.00,-24.93,6650.42
53568,2020,POLICE DEPARTMENT,09/27/2012,MANHATTAN,TRAFFIC ENFORCEMENT AGENT,ACTIVE,42377.0,per Annum,1929.50,41369.48,-2.00,-57.79,2238.91
55147,2020,POLICE DEPARTMENT,07/10/2002,MANHATTAN,POLICE ADMINISTRATIVE AIDE,ACTIVE,41988.0,per Annum,1638.00,33179.40,-1.00,-25.01,4695.83
55990,2020,POLICE DEPARTMENT,11/14/2005,BROOKLYN,SCHOOL CROSSING GUARD,ACTIVE,34626.0,per Annum,2080.00,34094.19,-5.00,-42.67,758.92
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4496522,2021,DEPARTMENT OF CORRECTION,01/16/1996,QUEENS,*COOK,ON LEAVE,41534.0,per Annum,400.00,-4602.61,-17.25,-543.13,-688.37
4496544,2021,ADMIN FOR CHILDREN'S SVCS,09/23/2019,BROOKLYN,YOUTH DEVELOPMENT SPECIALIST,CEASED,45759.0,per Annum,-280.00,-6163.62,-8.00,-200.37,-52.60
4496573,2021,DEPARTMENT OF SANITATION,04/03/1989,BROOKLYN,SANITATION WORKER,CEASED,81034.0,per Annum,0.00,0.00,-2.00,-39.09,-7504.64
4496634,2021,ADMIN FOR CHILDREN'S SVCS,02/25/2019,BROOKLYN,YOUTH DEVELOPMENT SPECIALIST,CEASED,49318.0,per Annum,-420.00,-10287.97,-24.50,-755.83,-21.87


In [30]:
# removing rows with negative value in all numerical columns as they can cause issues with calculation
df = df[(df['OT Hours'] >= 0) & \
        (df['Regular Gross Paid'] >= 0) & \
        (df['Total OT Paid'] >= 0) & \
        (df['Total Other Pay'] >= 0) & \
        (df['Regular Hours'] >= 0) ]

In [31]:
df.shape

(4339278, 13)

## Clean Agancy Start Date

In [32]:
df['Agency Start Date'].count()

4339278

In [33]:
df [df['Agency Start Date'] == '12/31/9999']

,Fiscal Year,Agency Name,Agency Start Date,Work Location Borough,Title Description,Leave Status as of June 30,Base Salary,Pay Basis,Regular Hours,Regular Gross Paid,OT Hours,Total OT Paid,Total Other Pay
160553,2020,DEPT OF ED PEDAGOGICAL,12/31/9999,MANHATTAN,TEACHER SPECIAL EDUCATION,ACTIVE,124909.00,per Annum,0.0,122242.80,0.0,0.0,36229.15
167797,2020,DEPT OF ED PEDAGOGICAL,12/31/9999,MANHATTAN,TEACHER,ACTIVE,124909.00,per Annum,0.0,122242.80,0.0,0.0,13012.34
168030,2020,DEPT OF ED PEDAGOGICAL,12/31/9999,MANHATTAN,TEACHER,ACTIVE,124909.00,per Annum,0.0,122242.80,0.0,0.0,12876.50
168250,2020,DEPT OF ED PEDAGOGICAL,12/31/9999,MANHATTAN,TEACHER,ACTIVE,124909.00,per Annum,0.0,122242.80,0.0,0.0,12759.82
168674,2020,DEPT OF ED PEDAGOGICAL,12/31/9999,MANHATTAN,TEACHER,ACTIVE,124909.00,per Annum,0.0,122242.80,0.0,0.0,12536.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4229761,2021,DEPT OF ED PARA PROFESSIONALS,12/31/9999,MANHATTAN,ANNUAL ED PARA,ACTIVE,35060.00,per Annum,0.0,34166.58,0.0,0.0,1891.84
4244083,2021,DEPT OF ED HRLY SUPPORT STAFF,12/31/9999,MANHATTAN,F/T SCHOOL AIDE,ACTIVE,17.04,per Hour,0.0,21309.88,0.0,0.0,8900.39
4271799,2021,DEPT OF ED HRLY SUPPORT STAFF,12/31/9999,MANHATTAN,F/T SCHOOL AIDE,ACTIVE,17.04,per Hour,0.0,14109.12,0.0,0.0,4764.70
4272592,2021,DEPT OF ED HRLY SUPPORT STAFF,12/31/9999,MANHATTAN,F/T SCHOOL AIDE,ACTIVE,17.04,per Hour,0.0,14040.96,0.0,0.0,4516.30


There are 415 rows with the extreme date of '12/31/9999' that we will remove first. The reason for this is that if we try to convert these values to datetime, the api returns an out of bounds error.

In [34]:
# Filter out 415 rows where date is '12/31/9999'

df = df[df['Agency Start Date'] != '12/31/9999']

In [35]:
df['Agency Start Date'].count()

4338865

In [36]:
df[df['Agency Start Date'] == ''].count()

Fiscal Year                   63
Agency Name                   63
Agency Start Date             63
Work Location Borough         63
Title Description             63
Leave Status as of June 30    63
Base Salary                   63
Pay Basis                     63
Regular Hours                 63
Regular Gross Paid            63
OT Hours                      63
Total OT Paid                 63
Total Other Pay               63
dtype: int64

Note: there are 63 blank values for Agency Start Date. Converting to datetime format will take care of deleting them.

In [37]:
# convert column to datetime in format yyyy-mm-dd

import pandas as pd

df['Agency Start Date']= pd.to_datetime(df['Agency Start Date'])

In [38]:
df['Agency Start Date'].count()

4338802

Now lets find any future date by comparing all the Agency Start Dates to the report date (i.e. the date as of the report - 06/30/2021)

In [39]:
import datetime

report_date = datetime.datetime(2021, 6, 30)

In [40]:
df[df['Agency Start Date'] > report_date].count()

Fiscal Year                   3
Agency Name                   3
Agency Start Date             3
Work Location Borough         3
Title Description             3
Leave Status as of June 30    3
Base Salary                   3
Pay Basis                     3
Regular Hours                 3
Regular Gross Paid            3
OT Hours                      3
Total OT Paid                 3
Total Other Pay               3
dtype: int64

There are 3 rows with future dates. We will remove them.

In [41]:
df = df[df['Agency Start Date'] <= report_date]

In [42]:
df['Agency Start Date'].count()

4338799

Now let us examine very old dates. Retirement age in NYC is 62 years. If we assume minimum working age as 18, then a person can be working for at most 44 years. Since our dataset includes fiscal data only started form 2014, then lets see how many records have data before 2014- - 44 = 1970

In [43]:
min_date = datetime.datetime(1970, 6, 30)

In [44]:
df[df['Agency Start Date'] < min_date].count()

Fiscal Year                   8721
Agency Name                   8721
Agency Start Date             8721
Work Location Borough         8721
Title Description             8721
Leave Status as of June 30    8721
Base Salary                   8721
Pay Basis                     8721
Regular Hours                 8721
Regular Gross Paid            8721
OT Hours                      8721
Total OT Paid                 8721
Total Other Pay               8721
dtype: int64

In [45]:
df[df['Agency Start Date'] < min_date]

,Fiscal Year,Agency Name,Agency Start Date,Work Location Borough,Title Description,Leave Status as of June 30,Base Salary,Pay Basis,Regular Hours,Regular Gross Paid,OT Hours,Total OT Paid,Total Other Pay
289,2020,OFFICE OF MANAGEMENT & BUDGET,1969-05-19,MANHATTAN,COMPUTER SPECIALIST,ACTIVE,82234.00,per Annum,1820.00,80971.40,0.0,0.0,4820.35
2687,2020,LAW DEPARTMENT,1969-03-10,MANHATTAN,CLERICAL ASSOCIATE,CEASED,41544.00,per Annum,607.03,13782.69,0.0,0.0,1642.20
4806,2020,POLICE DEPARTMENT,1962-04-27,MANHATTAN,EXECUTIVE AGENCY COUNSEL,ACTIVE,241116.00,per Annum,1820.00,237995.93,0.0,0.0,0.00
7565,2020,POLICE DEPARTMENT,1970-04-17,QUEENS,SURGEON DETAILED AS DEPUTY CHIEF SURGEON,ACTIVE,162293.00,per Annum,2080.00,158536.69,0.0,0.0,16772.36
35811,2020,POLICE DEPARTMENT,1966-12-16,MANHATTAN,CHAPLAIN,ACTIVE,72623.00,per Annum,2080.00,71499.86,0.0,0.0,6390.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4492286,2021,DEPT OF ED PER SESSION TEACHER,1968-10-03,MANHATTAN,TEACHER- PER SESSION,CEASED,33.18,per Day,0.00,0.00,0.0,0.0,0.32
4492299,2021,DEPT OF ED PER SESSION TEACHER,1970-03-10,MANHATTAN,TEACHER- PER SESSION,ACTIVE,33.18,per Day,0.00,0.00,0.0,0.0,0.32
4492469,2021,DEPT OF ED PER SESSION TEACHER,1968-09-06,MANHATTAN,TEACHER- PER SESSION,CEASED,33.18,per Day,0.00,0.00,0.0,0.0,0.26
4492685,2021,DEPT OF ED PER SESSION TEACHER,1967-09-08,MANHATTAN,TEACHER- PER SESSION,CEASED,33.18,per Day,0.00,0.00,0.0,0.0,0.15


This gives us 8741 rows. We will assume a buffer of 5 years and remove any rows older than 1970 - 5 = 1965

In [46]:
min_date = datetime.datetime(1965, 6, 30)

In [47]:
df[df['Agency Start Date'] < min_date].count()

Fiscal Year                   1711
Agency Name                   1711
Agency Start Date             1711
Work Location Borough         1711
Title Description             1711
Leave Status as of June 30    1711
Base Salary                   1711
Pay Basis                     1711
Regular Hours                 1711
Regular Gross Paid            1711
OT Hours                      1711
Total OT Paid                 1711
Total Other Pay               1711
dtype: int64

In [48]:
df[df['Agency Start Date'] < min_date]

,Fiscal Year,Agency Name,Agency Start Date,Work Location Borough,Title Description,Leave Status as of June 30,Base Salary,Pay Basis,Regular Hours,Regular Gross Paid,OT Hours,Total OT Paid,Total Other Pay
4806,2020,POLICE DEPARTMENT,1962-04-27,MANHATTAN,EXECUTIVE AGENCY COUNSEL,ACTIVE,241116.00,per Annum,1820.0,237995.93,0.0,0.0,0.00
94931,2020,HRA/DEPT OF SOCIAL SERVICES,1962-04-02,BROOKLYN,ADMIN JOB OPPORTUNITY SPEC NM,ACTIVE,81765.00,per Annum,1820.0,83356.85,0.0,0.0,4219.81
95663,2020,HRA/DEPT OF SOCIAL SERVICES,1963-09-16,MANHATTAN,SUPERVISOR I,ACTIVE,73524.00,per Annum,1820.0,72387.03,0.0,0.0,8570.99
96713,2020,HRA/DEPT OF SOCIAL SERVICES,1957-10-21,BROOKLYN,STAFF ANALYST,ACTIVE,67814.00,per Annum,1820.0,67447.53,0.0,0.0,5445.70
96942,2020,HRA/DEPT OF SOCIAL SERVICES,1963-10-07,MANHATTAN,ADMINISTRATIVE DIRECTOR OF SOCIAL SERVICES,CEASED,103509.00,per Annum,0.0,0.00,0.0,0.0,71334.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4484897,2021,DEPT OF ED PER SESSION TEACHER,1956-09-07,MANHATTAN,TEACHER- PER SESSION,CEASED,34.78,per Day,0.0,5.56,0.0,0.0,0.00
4485563,2021,DEPT OF ED PER SESSION TEACHER,1964-02-17,MANHATTAN,TEACHER- PER SESSION,CEASED,33.18,per Day,0.0,0.00,0.0,0.0,4.80
4488301,2021,DEPT OF ED PER SESSION TEACHER,1961-09-05,MANHATTAN,TEACHER- PER SESSION,CEASED,33.18,per Day,0.0,0.00,0.0,0.0,2.57
4488706,2021,DEPT OF ED PER SESSION TEACHER,1960-02-01,MANHATTAN,TEACHER- PER SESSION,CEASED,33.18,per Day,0.0,0.00,0.0,0.0,2.20


In [49]:
df['Agency Start Date'].count()

4338799

## Fix Title Descriptions 

Based on result of similarity analysis from the DataProfiling notebook, we have identified that this can be fixed by following steps -
  1. Trim the characters '?', '*', '-', and whitespaces from the names, this will fix issue with all but 6 clusters.
  2. For the remaining 6 clusters, we will fix it manually using the fixTitles() method below. 

In [50]:
df = update(df, columns='Title Description', func=lambda title:title.strip("?*- ").upper())

In [51]:
df.shape

(4338799, 13)

In [52]:
titles = df['Title Description'].unique()
len(titles)


1769

In [53]:
def fixTitles(title):
  if title == 'SERGEANTD/A SPECIAL ASSIGNMENT' or title == 'SERGEANT-D/A SPECIAL ASSIGNMENT':
    return 'SERGEANT D/A SPECIAL ASSIGNMENT'
  elif title == 'ASSISTANT DEPUTY COMMISSIONER':
        return 'DEPUTY ASSISTANT COMMISSIONER'
  elif title == 'CASE - MANAGEMENT NURSE':
        return 'CASE MANAGEMENT NURSE'
  elif title == 'SERGEANTD/A SUPERVISOR DETECTIVE SQUAD':
        return 'SERGEANT-D/A SUPERVISOR DETECTIVE SQUAD'
  elif title == 'ELECTRICIANS HELPER':
        return 'ELECTRICIAN\'S HELPER'
  elif title == 'GENERAL INSPECTOR':
        return 'INSPECTOR GENERAL'
  else:
    return title

In [54]:
df = update(df, columns='Title Description', func=fixTitles)

In [55]:
df.shape

(4338799, 13)

In [56]:
titles = df['Title Description'].unique()
len(titles)

1762

## Save Final Output

In [57]:
# check final count

df.count

<bound method DataFrame.count of         Fiscal Year  ... Total Other Pay
0              2020  ...               0
1              2020  ...               0
2              2020  ...               0
3              2020  ...               0
4              2020  ...               0
...             ...  ...             ...
4493888        2021  ...               0
4493890        2021  ...               0
4493927        2021  ...               0
4493982        2021  ...               0
4493984        2021  ...               0

[4338799 rows x 13 columns]>

We have finally removed approx 157,000 rows which represents approx 3% of the original data.

In [58]:
df.to_csv('DataCleaningOutput.csv', index=False)